<a href="https://colab.research.google.com/github/waltherwj/Tensorflow2-lectures/blob/master/TF2_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf


In [0]:
from tensorflow.keras.layers import Input, Dense, LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD, Adam

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

#scale data
x_train, x_test = (x_train/255.0)*2 - 1, (x_test/255.0)*2 -1
print("x_train.shape:", x_train.shape)

11493376/11490434 [==============================] - 0s 0us/step
x_train.shape: (60000, 28, 28)


In [8]:
N, H, W, = x_train.shape
D = H*W
x_train = x_train.reshape(-1,D)   #NOT CONV GAN
x_test = x_test.reshape(-1, D)
print(x_test.shape)

(10000, 784)


In [0]:
#hidden dim size
hidden_dim = 100

In [0]:
#get the generator model
def build_generator(latent_dim):
  i = Input(shape=(hidden_dim,))
  x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
  x = BatchNormalization(momentum=0.8)(x)
  x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
  x = BatchNormalization(momentum=0.8)(x)
  x = Dense(D, activation='tanh')(x)

  model = Model(i, x)

  return model


In [0]:
#get the discriminator model
def build_discriminator(img_size):
  i = Input(shape = (img_size,))
  x = Dense(512)(i)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dense(256)(x)
  x = LeakyReLU(alpha=0.2)(x)
  x = Dense(1, activation='sigmoid')(x)

  model = Model(i,x)

  return model








In [0]:
#build and compile discriminator
discriminator = build_discriminator(D)
discriminator.compile(
      loss = "binary_crossentropy",
      optimizer = Adam(learning_rate = 0.0002, beta_1=0.5),
      metrics = ['accuracy']
                        )

#build and compile generator

generator = build_generator(hidden_dim)



In [0]:
# create noise sample to be fed into generator
z = Input(shape=hidden_dim)
#for entry in range(z.shape[1]):
  #print(z[entry])

In [27]:
# generate image 
img = generator(z)
print(img)

# make sure only generator is trainable
discriminator.trainable = False

# get fake loss
fake_pred = discriminator(img)



Tensor("model_1_2/Identity:0", shape=(None, 784), dtype=float32)


In [0]:
#create the combined model
combined_model = Model(z, fake_pred)

In [0]:
#compile combined model
combined_model.compile(
    loss = 'binary_crossentropy',
    optimizer = Adam(0.0002, 0.5)
)

In [0]:
# Train the GAN

#CONFIG

batch_size = 32
epochs = 30000
sample_period = 200 #every sample period steps generate and save some data

# create batch labels to use when calling train_on_batch
ones = np.ones(batch_size)
zeros = np.zeros(batch_size)


#store the losses
g_losses = []
d_losses = []

#create a folder to store the generated images
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

In [0]:
# function to save sample images to image file
def sample_images(epoch):
  rows, cols = 5,5
  noise = np.random.randn(rows*cols, hidden_dim)
  imgs = generator.predict(noise)

  #rescale images 0-1

  imgs = 0.5* imgs + 0.5

  fig, axs = plt.subplots(rows, cols)
  idx = 0
  for i in range(rows):
    for j in range(cols):
      axs[i,j].imshow(imgs[idx].reshape(H,W), cmap = 'gray')
      axs[i,j].axis('off')
      idx += 1
  fig.savefig("gan_images/%d.png" % epoch)
  plt.close()

In [0]:
# Main Training Loop

for epoch in range(epochs):
  # Discriminator first

  # Select random batch of images
  idx = np.random.randint(low = 0, high = x_train.shape[0], size = batch_size)
  real_imgs = x_train[idx]

  #generate fake images
  noise = np.random.randn(batch_size, hidden_dim)
  fake_imgs = generator.predict(noise)
  
  #Train the driscriminator
  d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs,ones)
  d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)

  d_loss = 0.5 * (d_loss_real + d_loss_fake)
  d_acc = 0.5 * (d_acc_fake + d_acc_real)


  #train generator
  noise = np.random.randn(batch_size, hidden_dim)
  g_loss = combined_model.train_on_batch(noise, ones)
  
  #save losses

  d_losses.append(d_loss)
  g_losses.append(g_loss)

  if epoch % 100 == 0:
    print (f"epoch: {epoch +1}/{epochs}, d_loss: {d_loss:.2f},\
    d_acc: {d_acc:.2f}, g_loss: {g_loss:.2f}")

  if epoch % sample_period == 0:
    sample_images(epoch)



epoch: 1/30000, d_loss: 0.56,    d_acc: 0.53, g_loss: 0.52
epoch: 101/30000, d_loss: 0.01,    d_acc: 1.00, g_loss: 4.75
epoch: 201/30000, d_loss: 0.02,    d_acc: 1.00, g_loss: 4.64
epoch: 301/30000, d_loss: 0.08,    d_acc: 0.97, g_loss: 4.48
epoch: 401/30000, d_loss: 0.34,    d_acc: 0.86, g_loss: 2.14
epoch: 501/30000, d_loss: 0.52,    d_acc: 0.72, g_loss: 1.21
epoch: 601/30000, d_loss: 0.65,    d_acc: 0.58, g_loss: 0.92
epoch: 701/30000, d_loss: 0.59,    d_acc: 0.62, g_loss: 1.04
epoch: 801/30000, d_loss: 0.55,    d_acc: 0.75, g_loss: 0.99
